# JIT Copy Path File

### Documentation
* The purpose of this chunk of code is to transfer pytorch model([fastai saved](https://docs.fast.ai/callbacks.html#SaveModelCallback)) into pytorch jit model

* Install this command line ````sudo ln -s $HOME/ssdd/jitcp /usr/bin/jitcp````, assume you have this repository in your home directory

* ```chmod +x /usr/bin/jitcp```

* The transformation command line example:

````shell
jitcp --src="/home/paperspace/bestmodel_27.pth" --dst="jit_fpn_b6.pth" --model="fpn" --encoder="efficientnet-b6"

````

* ```src``` or ```dst``` could be relative or absolute path

* Prerequisite:
    * The process require a machine with **GPU CUDA**
    * Other libraries required for training the model

Experimental global variables for testing

In [1]:
# SOURCE  = "/home/paperspace/bestmodel_27.pth"
# DESTINATION  ="jit_fpn_b6.pth"
# MODELNAME = "fpn"
# ENCODER = "efficientnet-b6"

In [11]:
from argparse import ArgumentParser

Argument Parser for command line use

In [12]:
ap = ArgumentParser()

ap.add_argument("--src", dest = "src", help="file path for srouce weights, pth file")
ap.add_argument("--dst", dest = "dst", help="file path for destination weights, pth file")
ap.add_argument("--model", dest = "model", default ='fpn' , help="Model name either 'unet' or 'fpn, default 'fpn'")
ap.add_argument("--encoder", dest = "encoder", help='encoder backbone, like efficientnet-b6, efficientnet-b7')

args = ap.parse_args()

SOURCE = args.src
DESTINATION = args.dst
MODELNAME = args.model
ENCODER = args.encoder

In [2]:
from fastai.vision import *
import  segmentation_models_pytorch as smp
import torch

In [3]:
if MODELNAME == "fpn":
    modellib = smp.FPN
elif MODELNAME=="unet":
    modellib = smp.Unet
else:
    print("Didn't know about name %s"%(MODELNAME))
    exit(0)

In [4]:
print("Constructing model:\t%s\nWith Encoder Backbone:\t%s"%(MODELNAME, ENCODER))

Constructing model:	fpn
With Encoder Backbone:	efficientnet-b6


In [5]:
model = modellib(ENCODER,encoder_weights = None, classes = 4, activation=None)

In [6]:
print("Loading model to cuda")
model = model.cuda()

Loading model to cuda


In [7]:
print("Loading weights from:\t%s"%(SOURCE))

Loading weights from:	/home/paperspace/bestmodel_27.pth


In [8]:
model.load_state_dict(torch.load(SOURCE)["model"])

<All keys matched successfully>

In [9]:
testimg = torch.rand(2,3,256,256).cuda()
model = model.eval()

In [10]:
print("JIT tracing the model")
traced = torch.jit.trace(model, testimg)
traced.save(DESTINATION)
print("JIT model saved to:\t%s"%(DESTINATION))

JIT tracing the model
JIT model saved to:	jit_fpn_b6.pth
